In [1]:
#Import relevant packages
import sqlite3
import pandas as pd


In [8]:
#Change file path to the sql data table you want to explore
db_path = "/Users/tedwhite15/Documents/Taylers_Files/Research/Projects/Land-use/AVLU_project/GenX/PowerGenome/data/PowerGenome_data_repository/pg_misc_tables_efs_2023.2.sqlite/pg_misc_tables_efs_2023.2.sqlite"
conn = sqlite3.connect(db_path)



In [9]:
#This prints all of the tables inside this files 
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

# Print table names
print("Tables in database:")
for table in tables:
    print(table[0])




Tables in database:
load_curves_epaipm
load_curves_ferc
plant_region_map_epaipm
regions_entity_epaipm
technology_costs_nrelatb
transmission_single_epaipm
technology_heat_rates_nrelatb
load_curves_nrel_efs
load_curves_nrel_efs_state
offshore_spur_costs_nrelatb


In [10]:
# List columns in technology_costs_nrelatb
columns = pd.read_sql("PRAGMA table_info(technology_costs_nrelatb);", conn)
print(columns)

# Preview data
df = pd.read_sql("SELECT * FROM technology_costs_nrelatb LIMIT 10;", conn)
print(df)

    cid                name     type  notnull dflt_value  pk
0     0            atb_year  INTEGER        0       None   0
1     1          basis_year  INTEGER        0       None   0
2     2  cap_recovery_years  INTEGER        0       None   0
3     3           cost_case     TEXT        0       None   0
4     4      financial_case     TEXT        0       None   0
5     5          technology     TEXT        0       None   0
6     6         tech_detail     TEXT        0       None   0
7     7           parameter     TEXT        0       None   0
8     8     parameter_value    float        0       None   0
9     9               units     TEXT        0       None   0
10   10         dollar_year  INTEGER        0       None   0
   atb_year  basis_year  cap_recovery_years cost_case financial_case  \
0      2020        2050                  15      High         Market   
1      2019        2045                  15       Low         Market   
2      2019        2046                  15       Lo

In [15]:
# Look at the table technology_costs_nrelatb and limit to rows ghat correspond to atb_year = 2023

 pd.read_sql("""
SELECT DISTINCT technology, tech_detail, basis_year
FROM technology_costs_nrelatb
WHERE atb_year = 2023
ORDER BY technology, tech_detail, basis_year
""", conn)





,technology,tech_detail,basis_year
0,Battery,*,2021
1,Battery,*,2022
2,Battery,*,2023
3,Battery,*,2024
4,Battery,*,2025
...,...,...,...
4977,UtilityPV,Class9,2046
4978,UtilityPV,Class9,2047
4979,UtilityPV,Class9,2048
4980,UtilityPV,Class9,2049


In [22]:
# Look at the table technology_costs_nrelatb and limit to rows that correspond to atb_year = 2023
# and technology = Nuclear and basis_year = 2040


 pd.read_sql("""
SELECT DISTINCT technology, tech_detail, basis_year
FROM technology_costs_nrelatb
WHERE atb_year = 2023 AND technology = 'Nuclear' AND basis_year = 2040
ORDER BY tech_detail, basis_year
""", conn)

,technology,tech_detail,basis_year
0,Nuclear,*,2040
1,Nuclear,Nuclear,2040
2,Nuclear,NuclearSMR,2040


In [28]:
#Fo basis_year = 2040, check if there are any missing entries for costs

missing_costs = pd.read_sql("""
SELECT technology, tech_detail, parameter, basis_year, parameter_value
FROM technology_costs_nrelatb
WHERE technology = 'Nuclear' 
  AND basis_year = 2040
  AND parameter IN ('fixed_o_m_mw', 'variable_o_m_mwh', 'capex_mw')
ORDER BY tech_detail, parameter
""", conn)

missing_costs[missing_costs['parameter_value'].isna()]


,technology,tech_detail,parameter,basis_year,parameter_value


In [30]:
# List of parameters PowerGenome generally needs for new generators
required_params = [
    "capex_mw",        # capital cost per MW
    "fixed_o_m_mw",    # fixed O&M per MW
    "variable_o_m_mwh" # variable O&M per MWh
]

# Query Nuclear costs for 2040
query = f"""
SELECT tech_detail, parameter, parameter_value
FROM technology_costs_nrelatb
WHERE technology = 'Nuclear'
  AND basis_year = 2040
  AND parameter IN ({','.join(['?']*len(required_params))})
ORDER BY tech_detail, parameter
"""

df = pd.read_sql(query, conn, params=required_params)

# Pivot to see which parameters exist for each tech_detail
pivot = df.pivot_table(index='tech_detail', columns='parameter', values='parameter_value', aggfunc='first')

print(pivot)


parameter        capex_mw   fixed_o_m_mw  variable_o_m_mwh
tech_detail                                               
*            5.906304e+06  100888.671875          2.314453
Nuclear      6.373192e+06  144972.332016          2.351779
NuclearSMR   7.149832e+06  114000.000000          3.600000
